# ECDSA Coin

In [1]:
import json, uuid
from collections import namedtuple
from ecdsa import SigningKey, VerifyingKey, SECP256k1

In [2]:
bank_secret_exponent = 82131106019680303505331557872062346035977047103534656354347208601839669582133
bank_sk = SigningKey.from_secret_exponent(bank_secret_exponent, curve=SECP256k1)
bank_vk = bank_sk.get_verifying_key()
print("Bank's Private Number")
print(bank_sk.privkey.secret_multiplier)
print("Bank's Public Point")
bank_public_pair = f"{bank_vk.pubkey.point.x()},{bank_vk.pubkey.point.y()}"
print(bank_public_pair)

Bank's Private Number
82131106019680303505331557872062346035977047103534656354347208601839669582133
Bank's Public Point
95051501039409970067261119404617994246947706915280330673843616240644275979899,78071805429316904014428946848697643626369175962321260703189237284449878490278


In [3]:
john_secret_multiplier = 55717470057311173409666612548673609704723143884008771644960450831012091928340
john_sk = SigningKey.from_secret_exponent(john_secret_multiplier, curve=SECP256k1)
john_vk = john_sk.get_verifying_key()
print("John's Private Number")
print(john_sk.privkey.secret_multiplier)
print("John's Public Point")
john_public_pair = f"{john_vk.pubkey.point.x()},{john_vk.pubkey.point.y()}"
print(john_public_pair)

John's Private Number
55717470057311173409666612548673609704723143884008771644960450831012091928340
John's Public Point
87860107722650376950591458572346651373609029431396281467831473416418437360010,33742848381639124428566088663160518976625562547340001098886219967852176188186


In [4]:
minting_message = bank_public_pair.encode() + \
    b" issues 10 units to " + \
    john_public_pair.encode()

minting_signature = bank_sk.sign(minting_message)

note = [
    {
        "message": minting_message.decode(),
        "signature": minting_signature.hex(),
    }
]

json.dumps(note)

'[{"message": "95051501039409970067261119404617994246947706915280330673843616240644275979899,78071805429316904014428946848697643626369175962321260703189237284449878490278 issues 10 units to 87860107722650376950591458572346651373609029431396281467831473416418437360010,33742848381639124428566088663160518976625562547340001098886219967852176188186", "signature": "9e7382b0e258f13eeb8ad12ca2c67e1cf12d1f352d564ba5b0db1767818b5a36ca412886c0d7fa576035a869a20214abaa0cd70038c67f5affc7400efafc4fdd"}]'

In [5]:
jane_secret_multiplier = 10565481194119334483875173041211335223782298787713172586376122207587136082178
jane_sk = SigningKey.from_secret_exponent(jane_secret_multiplier, curve=SECP256k1)
jane_vk = jane_sk.get_verifying_key()
print("Jane's Private Number")
print(jane_sk.privkey.secret_multiplier)
print("Jane's Public Point")
jane_public_pair = f"{jane_vk.pubkey.point.x()},{jane_vk.pubkey.point.y()}"
print(jane_public_pair)

Jane's Private Number
10565481194119334483875173041211335223782298787713172586376122207587136082178
Jane's Public Point
101182973845320608199178831480609363769360760128553409660049830616697444913118,85294227196750083999328783434966373443271284891873157948555858416033959589163


In [6]:
john_to_jane_message = john_public_pair.encode() + \
    b" transfers to " + \
    jane_public_pair.encode()

john_to_jane_signature = john_sk.sign(john_to_jane_message)

transfer = {
    "message": john_to_jane_message.decode(),
    "signature": john_to_jane_signature.hex(),
}

new_note = note.copy()
new_note.append(transfer)

json.dumps(new_note)

'[{"message": "95051501039409970067261119404617994246947706915280330673843616240644275979899,78071805429316904014428946848697643626369175962321260703189237284449878490278 issues 10 units to 87860107722650376950591458572346651373609029431396281467831473416418437360010,33742848381639124428566088663160518976625562547340001098886219967852176188186", "signature": "9e7382b0e258f13eeb8ad12ca2c67e1cf12d1f352d564ba5b0db1767818b5a36ca412886c0d7fa576035a869a20214abaa0cd70038c67f5affc7400efafc4fdd"}, {"message": "87860107722650376950591458572346651373609029431396281467831473416418437360010,33742848381639124428566088663160518976625562547340001098886219967852176188186 transfers to 101182973845320608199178831480609363769360760128553409660049830616697444913118,85294227196750083999328783434966373443271284891873157948555858416033959589163", "signature": "de83a3a53b66a6a0e688a6cc4da7d346870a359f42735d649622cc78c0aceac652d26af98489200c9d613d0efa566ec2a89a21cc0a8e3ac791f6b9819d55cd94"}]'

In [7]:
len("20fbfb8ef6409de541c9a80ae008f013cffb80451ece0a71f7dd24816cafa453c2e2deade9a1f1ffbe1d1b58bb8593c7faed3ebacb56e019e6e60aafd1df4881")

128

In [8]:
import ecdsa

def validate_note(note_json):
    note = json.loads(note_json)
    
    minting_transfer = note[0]
    transfers = note[1:]
    
    minting_transfer_parts = minting_transfer["message"].split(" ")
    
    x, y = minting_transfer_parts[0].split(",")
    bank_point = ecdsa.ecdsa.ellipticcurve.Point(SECP256k1.curve, int(x), int(y))
    bank_vk = VerifyingKey.from_public_point(bank_point, curve=SECP256k1)
    verifies = bank_vk.verify(
        bytes.fromhex(minting_transfer["signature"]),
        minting_transfer["message"].encode(),
    )
    if not verifies:
        raise ValueError("Not minted by the bank")
    
    current_pair = minting_transfer_parts[-1]
    x, y = current_pair.split(",")
    current_point = ecdsa.ecdsa.ellipticcurve.Point(SECP256k1.curve, int(x), int(y))
    current_vk = VerifyingKey.from_public_point(current_point, curve=SECP256k1)
    
    for transfer in transfers:
        message_parts = transfer["message"].split(" ")
        next_pair = message_parts[0]

        if next_pair != current_pair:
            print("Transfer chain out-of-order")
        
        signature = minting_transfer_parts[-1]
        verifies = current_vk.verify(
            bytes.fromhex(transfer["signature"]),
            transfer["message"].encode(),
        )
        if not verifies:
            raise ValueError("Signature doesn't verify")
        
        current_pair = next_pair

validate_note(json.dumps(new_note))